# 2.2 Запросы на выборку, соединение таблиц

## Инициализация БД

In [1]:
%load_ext sql
import sqlalchemy
engine = sqlalchemy.create_engine('sqlite:///stepik.db')
%sql sqlite:///stepik.db

'Connected: @stepik.db'

In [2]:
# Подключение помощника по работе с учебными таблицами
import sys
sys.path.append('../helpers')
import tableHelper as th
th.setEngine(engine)

***

# Теория

## Один ко многим

Связь __«один ко многим»__ имеет место, когда одной записи главной таблицы соответствует несколько записей связанной таблицы, а каждой записи связанной таблицы соответствует только одна запись главной таблицы. Обозначается это так:

![](https://ucarecdn.com/3a1dee13-b46f-476e-82ee-d3c0a99ef729/)

__Этапы реализации связи «один ко многим»__ на следующем примере:

Один автор может написать несколько книг, а одна книга написана только одним автором. Для каждой книги известны ее количество и цена.

1. Создать таблицу `author`, в которую включить уникальных авторов книг, хранящихся на складе:  

![](https://ucarecdn.com/ebb2959d-32be-4d80-8855-abe8ce6ec4cb/)

2. Обе таблицы должны содержать первичный ключ, в таблице `book` он уже есть, в таблицу `author` добавим ключ `author_id`:

![](https://ucarecdn.com/88d82e42-3e5b-4e9a-b8e0-78f27f348b1b/)

3. Включить в таблицу `book` связанный столбец (внешний ключ, `FOREIGN KEY`), соответствующий по имени и типу ключевому столбцу главной таблицы (в нашем случае это столбец `author_id`). Для наглядности связь на схеме обозначается стрелкой от ключевого столбца главной таблицы к внешнему ключу связной таблицы:

![](https://ucarecdn.com/d504072b-bad1-4040-8f49-f5b7102fa1ca/)

## Многие ко многим

Связь __«многие ко многим»__ имеет место когда каждой записи одной таблицы соответствует несколько записей во второй, и наоборот, каждой записи второй таблицы соответствует несколько записей в первой. Обозначается это так:

![](https://ucarecdn.com/e53a7a38-0b4e-4705-b07e-209dd8074e49/)

__Этапы реализации связи «многие ко многим»__ на следующем примере:

Один автор может написать несколько книг, а одна книга может быть написана _несколькими_ авторами. Для каждой книги известны ее количество и цена.

1. Создать таблицу `author`, в которую включить уникальных авторов книг, хранящихся на складе:  

![](https://ucarecdn.com/3762c894-7798-4ff7-a254-4068635de34e/)

2. В обеих таблицах необходимо определить первичный ключ, в нашем случае в таблице `book` он уже есть, поэтому достаточно включить первичный ключ `author_id` в таблицу `author`:

![](https://ucarecdn.com/961abd2a-0a7c-42c4-ab52-b4e9cb2d9ceb/)

3.  Создать новую таблицу-связку, состоящую из двух столбцов, соответствующих по имени и типу ключевым столбцам исходных таблиц. Каждый из этих столбцов является внешним ключом (`FOREIGN KEY`)  и связан с ключевым столбцом каждой таблицы. Для наглядности связи на схеме обозначаются стрелкой от ключевого столбца исходной таблицы к внешнему ключу связной таблицы.

![](https://ucarecdn.com/6943bc53-bbe5-410b-9264-00b20eb0f017/)

4. Дальше необходимо определиться с первичным ключом таблицы-связки. Можно сделать два ключевых столбца, тогда все записи в этой таблице должны быть уникальными, то есть не повторяться. Для связи автор-книга этот вариант подходит. Но в некоторых случаях записи в таблице-связке могут повторяться, например, если мы будем продавать книги покупателям (один человек может купить несколько книг, а одну и ту же книгу могут купить несколько человек). Тогда в таблицу-связку включают дополнительные столбцы для идентификации записей, например, дату продажи, также в таблицу-связку добавляют первичный ключ. Мы воспользуемся вторым способом:

![](https://ucarecdn.com/9e4529dc-355a-4675-a782-a824fd11d4ce/)

# Упражнения

### Задание 1.

Создать таблицу `author` следующей структуры:

| Поле | Описание |
|:---:|:---:|
| author_id	| INT PRIMARY KEY AUTO_INCREMENT |
|name_author | VARCHAR(50) |

In [4]:
%%sql
CREATE TABLE author (
       author_id   INTEGER PRIMARY KEY,
       name_author VARCHAR(50))

 * sqlite:///stepik.db
Done.


[]

In [6]:
author = th.CSVfromDB('author', 'tables/author.csv')

### Задание 1

Заполнить таблицу `author`. В нее включить следующих авторов:
- Булгаков М.А.
- Достоевский Ф.М.
- Есенин С.А.
- Пастернак Б.Л.

| Поле | Описание |
|:---:|:---:|
| author_id	| INT PRIMARY KEY AUTO_INCREMENT |
|name_author | VARCHAR(50) |

In [8]:
%%sql
INSERT INTO author (name_author)
VALUES ("Булгаков М.А."),
       ("Достоевский Ф.М."),
       ("Есенин С.А."),
       ("Пастернак Б.Л.")

 * sqlite:///stepik.db
4 rows affected.


[]

In [10]:
author.save()

## Создание таблицы с внешними ключами `FOREIGN KEY`, `REFERENCES`

При создании зависимой таблицы (таблицы, которая содержит внешние ключи) необходимо учитывать, что :

каждый внешний ключ должен иметь такой же тип данных, как связанное поле главной таблицы (в наших примерах это `INT`);
необходимо указать главную для нее таблицу и столбец, по которому осуществляется связь:

>`FOREIGN KEY (связанное_поле_зависимой_таблицы)  
REFERENCES главная_таблица (связанное_поле_главной_таблицы)`

По умолчанию любой столбец, кроме ключевого, может содержать значение `NULL`. При создании таблицы это можно переопределить,  используя  ограничение `NOT NULL` для этого столбца:

>`CREATE TABLE таблица (
    столбец_1 INT NOT NULL, 
    столбец_2 VARCHAR(10) 
);`

В созданной таблице в `столбец_1` не может содержать пустое значение, а `столбец_2` - может.

Для внешних ключей рекомендуется устанавливать ограничение `NOT NULL` (если это совместимо с другими опциями, которые будут рассмотрены в следующем шаге).

__Пример__

Создать таблицу `book` следующей структуры:

| Поле | Тип, описание | Связи |
|:---:|:---:|:---:|
| book_id | INT PRIMARY KEY AUTO_INCREMENT |   |
| title | VARCHAR(50) |   |
| author_id | INT  | внешний ключ: главная таблица `author`, связанный столбец `author.author_id`, пустое значение не допускается |
| price | DECIMAL(8, 2) |   |
| amount | INT |   |

Запрос:

>`CREATE TABLE book (  
    book_id INT PRIMARY KEY AUTO_INCREMENT,  
    title VARCHAR(50),  
    author_id INT NOT NULL,  
    price DECIMAL(8,2),  
    amount INT,  
    FOREIGN KEY (author_id)  REFERENCES author (author_id)  
);`

### Задание 2

Перепишите запрос на создание таблицы `book`, чтобы ее структура соответствовала структуре, показанной на логической схеме (таблица `genre` уже создана, порядок следования столбцов - как на логической схеме в таблице `book`, `genre_id` - внешний ключ) . Для `genre_id` ограничение о недопустимости пустых значений не задавать. В качестве главной таблицы для описания поля  `genre_id` использовать таблицу `genre` следующей структуры:

| Поле | Тип, описание |
|:---:|:---:|
| genre_id | INT PRIMARY KEY AUTO_INCREMENT |
| name_genre | VARCHAR(30) |

In [11]:
%%sql
CREATE TABLE genre (
       genre_id   INTEGER PRIMARY KEY,
       name_genre VARCHAR(30))

 * sqlite:///stepik.db
Done.


[]

In [12]:
genre = th.CSVfromDB('genre', 'tables/genre.csv')

__Логическая схема__ (нужно создать только таблицу `book`):

![](https://ucarecdn.com/95045d96-412d-4e10-88f2-7ac6b13fada6/)

In [21]:
%%sql
DROP TABLE IF EXISTS book;

CREATE TABLE book (  
    book_id   INTEGER PRIMARY KEY,  
    title     VARCHAR(50),  
    author_id INTEGER NOT NULL,  
    genre_id  INTEGER,  
    price     DECIMAL(8,2),  
    amount    INTEGER,  
    FOREIGN KEY (author_id)  REFERENCES author (author_id),  
    FOREIGN KEY (genre_id)   REFERENCES genre (genre_id)  
);

PRAGMA table_info(book)

 * sqlite:///stepik.db
Done.
Done.
Done.


cid,name,type,notnull,dflt_value,pk
0,book_id,INTEGER,0,None,1
1,title,VARCHAR(50),0,None,0
2,author_id,INTEGER,1,None,0
3,genre_id,INTEGER,0,None,0
4,price,"DECIMAL(8,2)",0,None,0
5,amount,INTEGER,0,None,0


In [3]:
book = th.CSVfromDB('book')

<h2>Действия при удалении записи главной таблицы <code>ON DELETE</code></h2>

<p>С помощью выражения<strong> </strong><code>ON DELETE</code> можно установить действия, которые выполняются для записей подчиненной таблицы при удалении связанной строки из главной таблицы. При удалении можно установить следующие опции:</p>

<ul>
	<li><code>CASCADE</code>: автоматически удаляет строки из зависимой таблицы при удалении связанных строк в главной таблице.</li>
	<li><code>SET NULL</code>: при удалении  связанной строки из главной таблицы устанавливает для столбца внешнего ключа значение <strong><code>NULL</code></strong>. (В этом случае столбец внешнего ключа должен поддерживать установку<code><strong> NULL</strong></code>).</li>
	<li><code>SET DEFAULT</code> похоже на <code>SET NULL</code> за тем исключением, что значение  внешнего ключа устанавливается не в <code>NULL,</code> а в значение по умолчанию для данного столбца.</li>
	<li><code>RESTRICT</code>: отклоняет удаление строк в главной таблице при наличии связанных строк в зависимой таблице.</li>
</ul>

<p><strong>Важно! </strong>Если для столбца установлена опция <code>SET NULL</code>, то при его описании нельзя задать ограничение на пустое значение.</p>

<p><strong>Пример</strong></p>

<p>Будем считать, что при удалении автора из таблицы <code><strong>author</strong></code>, необходимо удалить все записи о книгах из таблицы <code><strong>book</strong></code>, написанные этим автором. Данное действие необходимо прописать при создании таблицы.</p>

<p><em>Запрос:</em></p>

<pre><code class="language-sql hljs"><span class="hljs-keyword">CREATE</span> <span class="hljs-keyword">TABLE</span> book (
    book_id <span class="hljs-built_in">INT</span> PRIMARY <span class="hljs-keyword">KEY</span> AUTO_INCREMENT, 
    title <span class="hljs-built_in">VARCHAR</span>(<span class="hljs-number">50</span>), 
    author_id <span class="hljs-built_in">INT</span> <span class="hljs-keyword">NOT</span> <span class="hljs-literal">NULL</span>, 
    price <span class="hljs-built_in">DECIMAL</span>(<span class="hljs-number">8</span>,<span class="hljs-number">2</span>), 
    amount <span class="hljs-built_in">INT</span>, 
    FOREIGN <span class="hljs-keyword">KEY</span> (author_id)  <span class="hljs-keyword">REFERENCES</span> author (author_id) <span class="hljs-keyword">ON</span> <span class="hljs-keyword">DELETE</span> <span class="hljs-keyword">CASCADE</span>
);</code></pre>

### Задание 3

Создать таблицу `book` той же структуры, что и на предыдущем шаге.  
Будем считать, что при удалении автора из таблицы `author`, должны удаляться все записи о книгах из таблицы `book`, написанные этим автором. 
А при удалении жанра из таблицы genre для соответствующей записи `book` установить значение `Null` в столбце `genre_id`. 

In [14]:
%%sql
DROP TABLE IF EXISTS book;

CREATE TABLE book (  
    book_id   INTEGER PRIMARY KEY,  
    title     VARCHAR(50),  
    author_id INTEGER NOT NULL,  
    genre_id  INTEGER,  
    price     DECIMAL(8,2),  
    amount    INTEGER,  
    FOREIGN KEY (author_id)  REFERENCES author (author_id) ON DELETE CASCADE,  
    FOREIGN KEY (genre_id)   REFERENCES genre (genre_id) ON DELETE SET NULL  
);

PRAGMA table_info(book)

 * sqlite:///stepik.db
Done.
Done.
Done.


cid,name,type,notnull,dflt_value,pk
0,book_id,INTEGER,0,None,1
1,title,VARCHAR(50),0,None,0
2,author_id,INTEGER,1,None,0
3,genre_id,INTEGER,0,None,0
4,price,"DECIMAL(8,2)",0,None,0
5,amount,INTEGER,0,None,0


### Задание 4

Для каждой строки таблицы `book` занесите значения в поля `author_id` и `genre_id`. Считать, что книга Есенина относится к жанру «Поэзия», остальные книги – к жанру «Роман».

Авторы и их произведения:

| Название книги | Автор | Цена | Количество |
|:---:|:---:|:---:|:---:|
| Мастер и Маргарита | Булгаков М.А. | 670.99 | 3 |
| Белая гвардия | Булгаков М.А. | 540.50 | 5 |
| Идиот | Достоевский Ф.М. | 460.00 | 10 |
| Братья Карамазовы | Достоевский Ф.М. | 799.01 | 3 |
| Игрок | Достоевский Ф.М. | 480.50 | 10 |
| Стихотворения и поэмы | Есенин С.А. | 650.00 | 15 |

In [9]:
author = th.Table('author')

In [7]:
book = th.Table('book')

In [16]:
book_init = th.Table('book_init', 'tables/book_init.csv', 'schemas/book_schema.csv')

In [19]:
book_init

__name__ = book_init
__path__ = tables/book_init.csv
 book_id                 title           author  price  amount
       1    Мастер и Маргарита    Булгаков М.А. 670.99       3
       2         Белая гвардия    Булгаков М.А. 540.50       5
       3                 Идиот Достоевский Ф.М. 460.00      10
       4     Братья Карамазовы Достоевский Ф.М. 799.01       3
       5                 Игрок Достоевский Ф.М. 480.50      10
       6 Стихотворения и поэмы      Есенин С.А. 650.00      15

In [6]:
%%sql
INSERT INTO book
SELECT book_id, title, 
       a.author_id,
       IIF(b.author = 'Есенин С.А.', 2, 1) AS genre_id,
       price, amount
  FROM book_init AS b, author AS a
 WHERE b.author = a.name_author

 * sqlite:///stepik.db
6 rows affected.


[]

In [8]:
book

__name__ = book
__path__ = tables/book.csv
Table in DataBase:
 book_id                 title  author_id  genre_id  price  amount
       1    Мастер и Маргарита          1         1 670.99       3
       2         Белая гвардия          1         1 540.50       5
       3                 Идиот          2         1 460.00      10
       4     Братья Карамазовы          2         1 799.01       3
       5                 Игрок          2         1 480.50      10
       6 Стихотворения и поэмы          3         2 650.00      15

In [10]:
genre = th.Table('genre')